In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for questions regarding geography. "
system_message += "Give short, friendly answers, no more than 2 sentences. You are currently still under construction, but\
you already have a lot of knowledge at your disposal. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
#Here are the knowledge lists:

country_capitals = {"france": "Paris", "belgium": "Brussels", "mali": "Bamako"}
country_populations = {"france": 68_170_000, "belgium": 11_820_000, "mali": 23_290_000}
capital_populations = {"france": 2_103_000, "belgium": 188_737, "mali": 4_228_000}

In [ ]:
def get_country_capital(country):
    print(f"Tool get_country_capital called for {country}")
    data_country = country.lower()
    return country_capitals.get(data_country, 'Unknown') #This is a default

In [ ]:
get_country_capital("Botswana")

In [ ]:
def get_country_population(country):
    print(f"Tool get_country_population called for {country}")
    data_country = country.lower()
    return str(country_populations.get(data_country, 'Unknown'))

In [ ]:
def get_capital_population(country):
    print(f"Tool get_capital_population called for {country}")
    data_country = country.lower()
    return str(capital_populations.get(data_country, 'Unknown'))

In [ ]:
country_capital_function = {
    "name": "get_country_capital",
    "description": "Get the capital city of a country. Call this whenever you need to know what a country's capital is, for example when a user asks 'What's the capital of this country?'",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "The country the user wants to know the capital of",
            },
        },
            "required": ["country"],
            "additionalProperties": False
    }
}

In [ ]:
country_population_function = {
    "name": "get_country_population",
    "description": "Get the population of a country. Call this whenever you need to know how many people live in a country, for example \
when a user asks 'What's the population of this country?'",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "The country the user wants to know the population of",
            },
        },
            "required": ["country"],
            "additionalProperties": False
    }
}

In [ ]:
capital_population_function = {
    "name": "get_capital_population",
    "description": "Get the population of a country's capital. Call this whenever you need to know how many people live in the capital of a country\
, for example when a user asks 'How many people live in the capital of this country?'",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "The country whose capital the user wants to know the population of",
            },
        },
            "required": ["country"],
            "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": country_capital_function}, {"type": "function", "function": country_population_function}, {"type": "function", "function": capital_population_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)
        messages.append(message)
        for response in responses:
            messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        indata = arguments.get("country") #in case of different functions with different numbers of arguments, this part will have to be 'abstracted'
        function_name = tool_call.function.name
        if function_name == 'get_country_capital':
            outdata = get_country_capital(indata)
        elif function_name == 'get_country_population':
            outdata = get_country_population(indata)
        else:
            outdata = get_capital_population(indata)
        
        responses.append({
            "role": "tool",
            "content": json.dumps({"country": indata,"outdata": outdata}),
            "tool_call_id": tool_call.id
        })
    return responses

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()